# Introduction

This notebook creates a  simulation of a Quantum Key Distribution (QKD) with a shared key between two parties (Alice and Bob). We consider the attack of a man in the middle (Eve, an eavesdropper that increases noise by adding a measure qubit or sabotages the QKD) and a parity qubit that checks if there's a third person. When the number of parity errors is greater than the threshold set by Alice and Bob, the QKD is not valid and they don't receive any key. 

Then we use the previous quantum circuit in blockchain. Now we have three parties (Alice, Bob and Charlie) that compete in a Byzantine Fault Tolerant (BFT) consensus. The competition works similar to rock paper scissors. A, B and C generates a one digit random number (0 or 1) and play until one of them gets a 1 and the rest a 0, so the first one wins. Each qubit is entangled to another qubit that reads the next player, i.e. A knows their results and C's, B knows the results from B and A and C knows C's and B's numbers. Then, A can certify C, B can certify A and C can certify B. 

However, A, B and C can cheat by modifying their maximally entangled qubit to be a pure state |1> (Cheat 1) or modifying the entangled qubit to be |0> (Cheat 2). Three parity qubits check whether someone cheated or not. There's consensus only when everyone plays fair or just one person cheats. If there is not consensus, the block is not validated and is restarted.

In [1]:
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import process_fidelity
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit.aqua.components.oracles import LogicalExpressionOracle, TruthTableOracle

import math

from qiskit.tools.visualization import plot_histogram

import numpy as np
import matplotlib.pyplot as plot

from qiskit import IBMQ
IBMQ.load_accounts()

# Symmetric quantum key distribution (QKD)

First we create the circuit to generate a symmetric QKD between Alice and Bob. There are three qubits: A, B and P, that is used to check parity between A and B qubits. This circuit is composed by four quantum gates. First we generate a random one-digit number (0 or 1) using a Hadamard gate on the first qubit and a CNOT gate targetting the second qubit. The man-in-the-middle sabotages or creates noise by adding a qubit to measure. This effect is modeled by a tensorial product of rotation matrices applying on each qubit with angles theta1 and theta2. When theta1 = theta2 = 0, Eve is not attacking. The parity gate sets P to 1 when there is not a parity error (A and B are equal), i.e. when there are not man-in-the-middle attacks or they are undetectable.  

The key is generated running the simulation N (number of shots) times and choosing the last num_bits (key's length) valid digits. In each shot the circuit checks if parity fails. When this happen, this shot is discarded automatically. If there are more parity fails than a security threshold set by A and B, the key is marked as invalid.
Our parity circuit cell:


In [ ]:

# ParityCircuit for random bit generator and parity bit
def parityCircuitCheating(cheating=False, cheatingType=0):

    if cheating:
        # Cheating operator
        c1 = cheatingMatrices()[cheatingType]
    else:
        c1 = np.identity(2)

    id_op = Operator(c1)

    truthtable = "10011001"
    oracle = TruthTableOracle(truthtable)
    or_cx = oracle.construct_circuit()
    # print(oracle.output_register)
    v = oracle.variable_register
    o = oracle.output_register

    cr1 = ClassicalRegister(3)
    cr2 = ClassicalRegister(1)

    cx_circ = QuantumCircuit(v, cr2)

    or_cx.add_register(cr1)
    cx_circ.h(v[1])
    cx_circ.cx(v[1], v[0])

    cx_circ.unitary(id_op, v[cheatingType+1:cheatingType+2], label='idop')


    total_cx = cx_circ + or_cx
    total_cx.measure(v, cr1)
    total_cx.measure(o, cr2)

    return total_cx

In [ ]:
x = np.identity(16)
theta = 0.01
x[0][0] = math.cos(theta)
x[1][0] = math.sin(theta)
x[0][1] = -math.sin(theta)
x[1][1] = math.cos(theta)

def rotationMatrix(theta):
    x = np.identity(2)
    x[0][0] = math.cos(theta)
    x[1][0] = math.sin(theta)
    x[0][1] = -math.sin(theta)
    x[1][1] = math.cos(theta)
    return x

Now we create the circuit

In [ ]:
  # ParityCircuit for random bit generator and parity bit
def parityCircuitCheating(cheating=False, cheatingType=0):

    if cheating:
        # Cheating operator
        c1 = cheatingMatrices()[cheatingType]
    else:
        c1 = np.identity(2)

    id_op = Operator(c1)

    truthtable = "10011001"
    oracle = TruthTableOracle(truthtable)
    or_cx = oracle.construct_circuit()
    # print(oracle.output_register)
    v = oracle.variable_register
    o = oracle.output_register

    cr1 = ClassicalRegister(3)
    cr2 = ClassicalRegister(1)

    cx_circ = QuantumCircuit(v, cr2)

    or_cx.add_register(cr1)
    cx_circ.h(v[1])
    cx_circ.cx(v[1], v[0])

    cx_circ.unitary(id_op, v[cheatingType+1:cheatingType+2], label='idop')


    total_cx = cx_circ + or_cx
    total_cx.measure(v, cr1)
    total_cx.measure(o, cr2)

    return total_cx


And using this circuit as the basic cell we perform the key generation:

In [ ]:
def generateQK(num_bits, theta1=0, theta2=0, securityThresh=1000, simulation=True, withHist=False):

    # Create the circuit
    cx_circ = parityCircuit(theta1, theta2)

    if simulation:
        # Execute the circuit
        print("Running on simulation...")
        job = execute(cx_circ, backend = Aer.get_backend('qasm_simulator'), shots=256*num_bits, memory=True)
        result = job.result()
    else:
        # Execute the circuit
        print("Running on real quantum computer...")
        job = execute(cx_circ, backend = IBMQ.get_backend('ibmqx2'), shots=256*num_bits, memory=True)
        result = job.result()

    # Print circuit.
    # print(cx_circ)

  
    # Print the result
    if withHist:
        counts = result.get_counts(cx_circ)
        plot.bar(counts.keys(), counts.values(), 1.0, color='g')
        plot.show()
    
    memory = result.get_memory()
    num_bits = int(num_bits)
    # memory = memory[len(memory)-num_bits: len(memory)]

    res = {'A': '', 'B': '', 'errorCounter':0, 'valid': True}
    counter = 0
    i = len(memory)-1
    while len(res["A"]) != num_bits:
        # print('Memory', memory[i], i)
        # Check if error in parity bit and discard if there is
        if memory[i][4] == '0':
            counter+=1
        else:
            res["A"] = res["A"] + memory[i][1]
            res["B"] = res["B"] + memory[i][2]
        
        # SecurityThreshold from which we discard the key
        if counter >= securityThresh:
            res['valid'] = False
            res["errorCounter"] = counter
            return res
        i-=1
        


    res["errorCounter"] = counter
    return res


# Quantum Rock-Paper Consensus (QRP Consensus) 

In Quantum Rock-Paper Consensus (QRP consensus), there are n (# of nodes) parties (A, B and C for n=3). Each one generates a random one-digit number (0 or 1) and they compete like in rock-paper-scissors but just rock-paper (two options). Each qubit is entangled to another one that reads the next player (A reads C, B reads. The game ends when one of them gets a 1 (paper) and the others get 0 (rock). Each game is represent the validation round in a blockchain algorithm. The winner is entitled for the next block, it is rewarded and consensus is reached.

We also introduce cheating. Each player can set their own qubit to 1 (cheat1), or their bitwise entangled qubit to 0 (cheat2). The code detects if someone cheats only if there is a parity error. Forbiding cheating or allowing cheat1, cheat2, both at the same time we have different probabilities to reach a consensus. When no one cheats, there is always consensus because there is no noise and qubits are maximally entangled so each player certifies the next one.
When only one type of cheat is allowed, no cheaters or just one cheater the system still leads to consensus. 
When there are two cheaters, there is still 1/4 probability to reach consensus.
And when all of the nodes cheat, there is only 1/8 to reach a correct consensus. We still achieve better potential consensus that existing BFT consensus.

This quantum circuit uses QKD circuit n (# of nodes) times. The number of qubits is 3n.

In [ ]:

# ParityCircuit for random bit generator and parity bit
def parityCircuitCheating(cheating=False, cheatingType=0):

    if cheating:
        # Cheating operator
        c1 = cheatingMatrices()[cheatingType]
    else:
        c1 = np.identity(2)

    id_op = Operator(c1)

    truthtable = "10011001"
    oracle = TruthTableOracle(truthtable)
    or_cx = oracle.construct_circuit()
    # print(oracle.output_register)
    v = oracle.variable_register
    o = oracle.output_register

    cr1 = ClassicalRegister(3)
    cr2 = ClassicalRegister(1)

    cx_circ = QuantumCircuit(v, cr2)

    or_cx.add_register(cr1)
    cx_circ.h(v[1])
    cx_circ.cx(v[1], v[0])

    cx_circ.unitary(id_op, v[cheatingType+1:cheatingType+2], label='idop')


    total_cx = cx_circ + or_cx
    total_cx.measure(v, cr1)
    total_cx.measure(o, cr2)

    return total_cx


Quantum consensus algorithm:

In [ ]:
def quantumConsensus(nodes=3, cheating=False, cheatingType=[0,0,0]):

    # Placeholder for the nodes results
    res = dict(zip(string.ascii_uppercase, range(1, nodes+1)))
    toSend = dict(zip(string.ascii_uppercase, range(1, nodes+1)))

    

    # While two nodes with the same value
    while sum(res.values()) != 1:
        # Placeholder for all measurements
        pkts = ''
        for i in range(nodes):
            cx_circ = parityCircuitCheating(cheating, cheatingType[i])
            job = execute(cx_circ, backend = Aer.get_backend('qasm_simulator'), shots=1, memory=True)
            result = job.result()
            memory = result.get_memory()
            res[list(res.keys())[i]] = int(memory[0][1])

            #Adding measurements
            pkts += memory[0][2]
            pkts += memory[0][1]
            print(res)
    
    print(res)
    tmp = res.copy()
    for value in tmp:
        if res[value] == 1:
            res['winner'] = value 

    # Corner case
    print(pkts)
    toSend[list(toSend.keys())[0]] = pkts[0] + pkts[len(pkts)-1]
    #Adding proper nodes
    for i in range(0, nodes):
        # Preparing the packets
        toSend[list(toSend.keys())[i]] = pkts[2*i] + pkts[2*i-1]

    return res, toSend


Example of consensus using two nodes.

In [ ]:
def network3nodes(cheating, cheatingType):
    res = quantumConsensus(3, cheating, cheatingType)
    toSend = res[1]

    #TODO: Check the mistake here in the validation.
    # Cheat check
    if toSend['A'][1] == toSend['C'][0]:
        print('As validation correct')
    else:
        print('As validation failed. Someone cheating')

    if toSend['B'][1] == toSend['A'][0]:
        print('Bs validation correct')
    else:
        print('Bs validation failed. Someone cheating')

    if toSend['C'][1] == toSend['B'][0]:
        print('Cs validation correct')
    else:
        print('Cs validation failed. Someone cheating')
    